# 2. Application setup - Creating a boilerplate Q&A app

In this tutorial, you create a functional question-and-answer application using the PhariaAI Custom Applications quick-start guide. Your interactive Q&A tool will respond to natural language queries.

The Q&A app you build acts as the user-facing layer of your RAG system, connecting the document index with large language models to provide contextually relevant answers based on your specific document collection.

## Application components

The PhariaAI Q&A app architecture consists of several interconnected elements:

- **PhariaAI CLI**: Command-line interface for scaffolding, creating, and managing applications
- **Configuration files**: Define how your app accesses document collections and language models
- **User interface**: Provides a web-based interface for users to submit questions and receive answers
- **Service layer**: Handles request processing and orchestrates the retrieval and generation workflow
- **Skill component**: Implements the core RAG logic that retrieves documents and generates answers

## What you will do next

1. Set up a new Q&A app using the PhariaAI CLI with the specialised `qa` template
2. Configure the app components to access your document collection
3. Customise the app settings for optimal retrieval and response performance
4. Deploy your app for local testing and production environments

## Prerequisites

Before starting, ensure your PhariaAI installation satisfies the following prerequisites:

- **PhariaAI version**: You are running PhariaAI version 0.52.4 or higher
- **Custom Skills support**: Enabled in your PhariaAI instance with default PhariaKernel namespace names; and you have the credentials for the OCI registry
- **Custom applications support**: Enabled in your PhariaAI instance and you have the credentials for the OCI registry
- **Platform components**: You have enabled PhariaAssistant, PhariaOS Manager, PhariaKernel, and PhariaStudio, and you have the URLs to access them

As a PhariaAI developer, you also need:

- **API token**: A valid Aleph Alpha API token with appropriate permissions
- **Component URLS**: You have the URLs to access PhariaAssistant, PhariaKernel, and PhariaStudio
- **Permissions**: The *StudioUser* permission, as described in [User Setup](1.%20Introduction%20-%20Getting%20Started.ipynb#user-setup)

### 1. Create a new custom app in PhariaAI

The first step is to create the application scaffold using the PhariaAI CLI. This tool automatically generates the necessary files and folder structure for a complete app. The generated app includes the main components:

* a user interface
* a backend service
* a Skill that contains the core app logic

Run the following command in your terminal:

In [ ]:
npx @aleph-alpha/pharia-ai-cli create

The CLI prompts you to provide configuration information for your new application. First, it will ask for an application name:

```python
✔ Please provide a new app name › <your-app-name>
```

Enter the name: "rag_tutorial".

Next, the CLI asks you to select a template. Templates provide pre-configured application structures optimised for different use cases:

```python
✔ Please select a template › base / summary / qa
```

Select the `qa` template.

### 2. Set up application components

After creating the application scaffold, we need to set up each component individually. The application consists of three main parts that operate together:
- the user interface (UI)
- the service, that is, the application backend
- the Skill, that is, the application logic

Each component requires its own dependencies to be installed and its environment to be configured.

We need to use separate terminal sessions for each component, as the components run simultaneously during development.

#### 2.1 Configure the application UI

The UI component provides the web interface that users interact with. It's built using modern web technologies and needs to be configured with authentication to communicate with the PhariaAI platform.

**Note**: To get a token, see the PhariaAI documentation article: [How to obtain a bearer token in PhariaStudio](https://docs.aleph-alpha.com/products/pharia-ai/pharia-studio/tutorial/add-intelligence-layer/#set-your-environment-variables).

In your terminal, enter the following commands:

In [ ]:
cd <your-app-name>/ui
pnpm install

Before starting the UI, you need to set up authentication. Open the environment file and set your bearer token in `<your-app-name>/ui/.env.` with the identifier `VITE_USER_TOKEN=<your-bearer-token>`

Now start the UI development server:

In [ ]:
pnpm run dev

#### 2.2 Configure the application service

The service component functions as the backend for your application. It handles API requests from the UI, manages application state, and communicates with the Skill component to process user queries. It's built with Python and runs as a web server.

In your terminal, enter the following commands:

In [ ]:
cd <your-app-name>/service

**Important**: Before setting up the service, ensure your environment is properly configured:

- Ensure your local Python version matches the one specified in pyproject.toml. For example, you can use the command `pyenv install x.x.x && pyenv local x.x.x`; see https://github.com/pyenv/pyenv
- Verify that any missing environment variables are set in <your-app-name>/service/.env

Now install the dependencies and start the service:

In [ ]:
uv sync --dev
uv run dev

#### 2.3 Configure the application Skill

The Skill component contains the core logic of your application, including document retrieval and answer generation. Unlike the UI and service components, the Skill is designed to be executed by the PhariaKernel platform rather than running directly on your local machine. However, we need to set up the development environment locally.

First, configure the environment variables for the OCI registry, which are used to publish the skill. This refers to the .env file in the project root:

```bash
SKILL_REGISTRY=registry.your-organisation.com
SKILL_REPOSITORY=your-pharia-skills-repository
SKILL_REGISTRY_USER=your-registry-username
SKILL_REGISTRY_TOKEN=your-registry-password
```

Navigate to the skill directory and set up the environment:

In [ ]:
cd <your-app-name>/skill

Load the environment variables and install dependencies:

In [ ]:
set -a && source ../.env
uv sync --dev

#### 2.4 Conclusion 

Congratulations! You now have your first custom PhariaAI application up and running. To see your application in action, simply visit the application UI URL. The user interface is already set up to communicate with the application service, and the application service is configured to work with the application Skill.

**Note**: API calls from the application service to the Skill will fail! This is because the Skill needs to be served through PhariaKernel. To resolve this, you publish the Skill so that it can be served through your PhariaKernel instance.


### 3. Publish the Skill

Although your application is running locally, there remains a critical step to cover before it is fully functional. Currently, the Skill component is only accessible locally; it is not yet available to PhariaKernel. To solve this, we need to build and publish the Skill to the registry, making it available for production use.


In your terminal, ensure you are in the Skill folder (for example, enter `cd <your-app-name>/skill`). First, you build the Skill. This process packages your code and dependencies into a container that can be deployed on PhariaKernel:

In [ ]:
cd <your-app-name>/skill
uv run pharia-skill build qa

Now, you are asked to publish the Skill to the registry. This makes it accessible to the PhariaKernel service.

You are asked to provide both a name and a tag for your Skill when publishing it. For this example, we choose the name ```qa-rag-tutorial```

**Note**:  Ensure that your chosen Skill name is unique. If you use an existing Skill's name, the Skill with that name will be replaced!

The response looks something like this:

```bash
╭────────────────────────────────── Publish Successful ──────────────────────────────────╮
│                                                                                        │
│  Skill: qa-rag-tutorial                                                                │
│  Tag: latest                                                                           │
│  Registry: <your-registry>/skills                                                      │
│                                                                                        │
│  Published in 11.14 seconds                                                            │
│                                                                                        │
╰────────────────────────────────────────────────────────────────────────────────────────
```

The last step is to configure PhariaKernel to recognise and serve your Skill. Update your ```namespace.toml``` file (ask your PhariaAI Operator for the location of this file) by adding the new Skill entry. Once updated, your application is ready to use the published Skill.

### 4. Adjust the routing configuration

After publishing your Skill, you need to connect your application's service component to the published Skill. This requires updating the routing configuration in your service component to point to the correct namespace and Skill name.

The service component acts as a bridge between the UI and the Skill, passing requests and responses between them. By default, the service is configured to use a Skill named "qa" in the "app" namespace, but we need to update this to match our published Skill.

To be able to connect the UI to the Skill you just published, we adjust the `<Application-name>/service/source/routes.py` file to point to the correct namespace and skill. In this file, you need to update the following line of code:

```python
skill = Skill(namespace="app", name="qa")

In most of our deployments, the namespace in which PhariaKernel is running is called "playground", and the Skill has the name you gave it. So in our example, we need to replace the above line with:

```python
skill = Skill(namespace="playground", name="qa-rag-tutorial")

As we have not adjusted the Skill yet, it still points to the example collection we set up, which contains papers regarding transformers. Although we will change this in the next part of the tutorial, when previewing your application now, you can ask questions like: "What is an encoder?"

After these changes, you need to restart the service component to point to the changed namespace and Skill. First, enter Ctrl-C in your terminal to stop the service, and then start it again with this:

In [ ]:
uv sync --dev
uv run dev

### 5. Preview your application

With your application components set up, Skill published, and routing configured, you can now preview and test your application. The PhariaAI platform provides tools for previewing applications locally before making them available to all users.

The preview process allows you to test your application in a development environment, ensuring everything is working correctly before deploying to production. During preview, your application runs locally but connects to the published Skill through PhariaKernel.

**Note**: Before using the `pharia-ai-cli` command below, ensure all necessary environment variables are configured correctly in `<your-app-name>/.env`.

To preview your application, run the following command:

In [ ]:
cd <your-app-name>
npx @aleph-alpha/pharia-ai-cli preview

This command starts your application in a local development mode and provides a URL that you can add to PhariaAssistant for testing:

Copy the preview URL and follow the instructions in [How to use developer mode in PhariaAssistant](https://docs.aleph-alpha.com/products/pharia-ai/pharia-assistant/use-dev-mode/). (You need to add the URL to the Developer Mode Server URL list in PhariaAssistant.)

Your application now appears in PhariaAssistant's left sidebar.

### 6. Conclusion

In this section, you successfully created and deployed a PhariaAI Q&A application that can answer questions based on your document collection:

✅ **Created the application scaffold** using the PhariaAI CLI with the `qa` template, which generated:
   - A responsive user interface for submitting questions
   - A backend service to handle requests
   - A Skill component that implements the RAG logic

✅ **Set up and configured all components**:
   - Installed dependencies for each component
   - Set up authentication and environment variables
   - Started each component in development mode

✅ **Published the Skill to the registry**:
   - Built the Skill container with all necessary dependencies
   - Published it to the OCI registry for use by PhariaKernel
   - Updated your service configuration to point to the published Skill

✅ **Previewed and tested the application**:
   - Started the application in preview mode
   - Connected it to PhariaAssistant for testing
   - Verified proper document retrieval and answer generation

Your Q&A application now provides a complete interface for users to ask questions about your document collection and receive contextually relevant answers generated by the language model while being grounded in your documents.

In the next section, you will customise your Skill to connect to your specific document collection and finetune its behaviour for your use case.